### TODO Idea
1. add scale to location net
2. initial h_t should come from images
### Source
[paper](https://arxiv.org/pdf/1406.6247.pdf)
[ref](https://github.com/kevinzakka/recurrent-visual-attention)

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision import transforms
from matplotlib import pyplot as plt
from torchvision import transforms
import torch.nn.functional as F
from torch.autograd import Variable

pilImage = transforms.ToPILImage()

from models import GlimpseSensor, GlimpseNetwork, CoreNetwork, ActionNetwork, LocationNetwork, BaselineNetwork, RecurrentAttention

### Load Images

In [3]:
mnist_trainset = datasets.MNIST('../../data', train=False, download=True,
                                   transform=transforms.Compose([
                                       transforms.ToTensor(),
#                                        transforms.Normalize((0.1307,), (0.3081,))
                                   ]))
data_loader_train = DataLoader(mnist_trainset, 64, shuffle=True, num_workers=8)

In [4]:
data, target = next(iter(data_loader_train))
pilImage(data[0])

### Test GlimpseSensor

In [5]:
mock_location = torch.Tensor((1, 0)).repeat(1,64).view(64, 2)
g = GlimpseSensor(retina_size=28)
a = g.foveate(data, locations=mock_location)
a = a.view(-1, 3, 1, 28, 28) # (batch, num_zoom_image, channel, H, W)
pilImage(a[0][0])

### GlimpseNetwork

In [6]:
glimseNetwork = GlimpseNetwork(
                     num_hidden_glimpse = 128,
                     num_hidden_location = 128,
                     retina_size = 7,
                     num_zoom_image = 3,
                     scale = 2,
                     channel = 1)
g_t = glimseNetwork(images=data, location_prev=mock_location)
g_t

tensor([[0.0599, 0.1613, 0.0606,  ..., 0.0773, 0.0487, 0.0461],
        [0.0779, 0.1181, 0.0475,  ..., 0.0318, 0.0224, 0.0384],
        [0.0413, 0.1010, 0.0489,  ..., 0.0229, 0.0121, 0.0384],
        ...,
        [0.0768, 0.1010, 0.0807,  ..., 0.0383, 0.0114, 0.0384],
        [0.0472, 0.1010, 0.0793,  ..., 0.0415, 0.0000, 0.0384],
        [0.0270, 0.1114, 0.0366,  ..., 0.0173, 0.0081, 0.0384]],
       grad_fn=<ReluBackward>)

### Core Network

In [7]:
h_prev = torch.zeros((256))

In [8]:
coreNetwork = CoreNetwork(num_hidden_glimpse_n_location = 256, num_hidden_core=256)
h = coreNetwork(g_t = g_t, h_prev = h_prev)
h.shape

torch.Size([64, 256])

### ActionNetwork

In [9]:
actionNetwork = ActionNetwork(num_hidden_core=h.shape[1], num_action=10)
actionNetwork(h).shape

torch.Size([64, 10])

### LocationNetwork

In [10]:
locationNetwork = LocationNetwork(num_hidden_core=h.shape[1], num_location=2, std=0.17)
mu, location = locationNetwork(h)
mu.shape, location.shape

(torch.Size([64, 2]), torch.Size([64, 2]))

### BaselineNetwork

In [11]:
baselineNetwork = BaselineNetwork(num_hidden_core=h.shape[1], num_output=1)
b = baselineNetwork(h)
b.shape

torch.Size([64, 1])

### RecurrentAttention

In [59]:
num_glimpse = 6
ram = RecurrentAttention(
     num_hidden_glimpse = 128,
     num_hidden_location = 128,
     num_hidden_core = 256,
     num_zoom_image = 3,
     num_glimpse=num_glimpse,
     num_action=10,
     retina_size = 7,
     scale = 2,
     channel = 1,
     std=0.17)
ram(data)

(tensor([[-2.2331, -2.3480, -2.2708, -2.1502, -2.3336, -2.3978, -2.4014, -2.3374,
          -2.3208, -2.2603],
         [-2.2457, -2.3243, -2.2771, -2.1306, -2.3264, -2.3959, -2.4125, -2.3338,
          -2.3250, -2.2837],
         [-2.2297, -2.3303, -2.2958, -2.1609, -2.3337, -2.3894, -2.3808, -2.3384,
          -2.2964, -2.2918],
         [-2.2351, -2.3136, -2.2851, -2.1460, -2.3422, -2.4032, -2.4107, -2.3308,
          -2.3193, -2.2679],
         [-2.2395, -2.3388, -2.2813, -2.1696, -2.3305, -2.3894, -2.3934, -2.3429,
          -2.2919, -2.2700],
         [-2.2478, -2.3211, -2.2834, -2.1419, -2.3465, -2.4044, -2.3954, -2.3317,
          -2.3121, -2.2681],
         [-2.2252, -2.3400, -2.2815, -2.1786, -2.3178, -2.3966, -2.4104, -2.3335,
          -2.3009, -2.2645],
         [-2.2337, -2.3248, -2.2859, -2.1557, -2.3318, -2.4358, -2.4200, -2.3041,
          -2.3040, -2.2608],
         [-2.2318, -2.3526, -2.2671, -2.1456, -2.3408, -2.4043, -2.4045, -2.3305,
          -2.3033, -2.2743],
 

### Train

In [65]:
optimizer = torch.optim.Adam(ram.parameters(), lr=3e-3)
for t in range(1000):
    data, y = next(iter(data_loader_train))
    log_probas, log_pis, baselines = ram(data)
    
    # reward
    predicted = torch.max(log_probas, 1)[1]
    correct = (predicted.detach() == y).float() # detach()????
    R = correct.unsqueeze(1).repeat(1, num_glimpse)
    
    adjusted_reward = R - baselines.detach()
    loss_reinforce = torch.sum(-log_pis*adjusted_reward, dim=1)
    loss_reinforce = torch.mean(loss_reinforce, dim=0)
    loss_action = F.nll_loss(log_probas, y)
    loss = loss_reinforce + loss_action

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    
    acc = 100 * (correct.sum() / len(y))
    print('acc:', acc.numpy(), 'loss:', loss.detach().numpy())

acc: 40.625 loss: -0.5688647
acc: 34.375 loss: -0.49798656
acc: 37.5 loss: -0.14341629
acc: 39.0625 loss: 0.17888749
acc: 42.1875 loss: -0.23501134
acc: 48.4375 loss: -0.9720911
acc: 45.3125 loss: -0.78758967
acc: 53.125 loss: -1.467391
acc: 48.4375 loss: -1.2042782
acc: 48.4375 loss: -1.0110687
acc: 45.3125 loss: -0.50475967
acc: 59.375 loss: -1.2751955
acc: 45.3125 loss: -0.73973155
acc: 43.75 loss: -0.11872089
acc: 39.0625 loss: -0.27893305
acc: 40.625 loss: -0.39890087
acc: 48.4375 loss: -0.73813975
acc: 39.0625 loss: -0.47749007
acc: 40.625 loss: -0.5876181
acc: 42.1875 loss: -0.55774856
acc: 26.5625 loss: 0.48112142
acc: 48.4375 loss: -1.0298237
acc: 59.375 loss: -1.3511432
acc: 59.375 loss: -1.1670583
acc: 53.125 loss: -0.75354505
acc: 43.75 loss: -0.707415
acc: 35.9375 loss: -0.20566821
acc: 48.4375 loss: -0.9625201
acc: 57.8125 loss: -1.5450945
acc: 50.0 loss: -0.87034965
acc: 46.875 loss: -0.79112124
acc: 51.5625 loss: -1.0812579
acc: 40.625 loss: -0.30816364
acc: 42.1875 los

acc: 85.9375 loss: -3.8197045
acc: 75.0 loss: -3.0210552
acc: 60.9375 loss: -2.3601437
acc: 78.125 loss: -3.071231
acc: 76.5625 loss: -2.5714533
acc: 62.5 loss: -1.8964741
acc: 75.0 loss: -3.2321262
acc: 73.4375 loss: -2.577521
acc: 81.25 loss: -3.801052
acc: 75.0 loss: -3.0145419
acc: 76.5625 loss: -3.1034985
acc: 84.375 loss: -3.815574
acc: 79.6875 loss: -2.8107805
acc: 70.3125 loss: -2.8028893
acc: 79.6875 loss: -3.2939477
acc: 75.0 loss: -2.8197346
acc: 67.1875 loss: -2.3180757
acc: 70.3125 loss: -2.6152248
acc: 71.875 loss: -2.5712793
acc: 71.875 loss: -2.731742
acc: 70.3125 loss: -2.2538214
acc: 75.0 loss: -2.9627929
acc: 76.5625 loss: -2.6275678
acc: 70.3125 loss: -2.4489522
acc: 76.5625 loss: -2.8149827
acc: 76.5625 loss: -2.754521
acc: 65.625 loss: -2.312756
acc: 73.4375 loss: -2.9990396
acc: 87.5 loss: -3.531276
acc: 70.3125 loss: -2.867074
acc: 78.125 loss: -3.300788
acc: 70.3125 loss: -2.485016
acc: 84.375 loss: -4.06205
acc: 79.6875 loss: -3.0915222
acc: 76.5625 loss: -2.7

acc: 82.8125 loss: -3.235753
acc: 82.8125 loss: -3.1869667
acc: 81.25 loss: -2.8483298
acc: 76.5625 loss: -2.4414973
acc: 71.875 loss: -2.3785276
acc: 75.0 loss: -2.6879172
acc: 85.9375 loss: -3.0642278
acc: 84.375 loss: -3.7201211
acc: 87.5 loss: -3.4902802
acc: 84.375 loss: -3.8912992
acc: 82.8125 loss: -4.0397916
acc: 87.5 loss: -4.303521
acc: 76.5625 loss: -3.0523815
acc: 81.25 loss: -4.435495
acc: 89.0625 loss: -4.3479257
acc: 84.375 loss: -3.9780164
acc: 89.0625 loss: -3.2950783
acc: 90.625 loss: -3.247013
acc: 81.25 loss: -2.9412956
acc: 81.25 loss: -3.1159625
acc: 82.8125 loss: -3.00007
acc: 78.125 loss: -2.895609
acc: 87.5 loss: -2.777152
acc: 84.375 loss: -3.7405748
acc: 84.375 loss: -3.96528
acc: 89.0625 loss: -3.7071743
acc: 89.0625 loss: -4.332907
acc: 82.8125 loss: -4.1254306
acc: 81.25 loss: -3.0721207
acc: 79.6875 loss: -3.3461199
acc: 79.6875 loss: -3.484336
acc: 82.8125 loss: -3.1375518
acc: 87.5 loss: -3.7670348
acc: 85.9375 loss: -3.650989
acc: 76.5625 loss: -2.8293

acc: 87.5 loss: -3.601067
acc: 93.75 loss: -4.1231465
acc: 85.9375 loss: -3.1557076
acc: 90.625 loss: -3.9553368
acc: 87.5 loss: -2.9513798
acc: 89.0625 loss: -4.098056
acc: 81.25 loss: -3.1338832
acc: 87.5 loss: -3.5698006
acc: 87.5 loss: -4.3907623
acc: 87.5 loss: -4.314297
acc: 89.0625 loss: -3.8715365
acc: 84.375 loss: -3.326603
acc: 84.375 loss: -3.1291974
acc: 87.5 loss: -3.455706
acc: 85.9375 loss: -3.8601267
acc: 98.4375 loss: -4.203083
acc: 95.3125 loss: -4.485583
acc: 87.5 loss: -4.3818836
acc: 89.0625 loss: -4.0287056
acc: 82.8125 loss: -3.5727122
acc: 92.1875 loss: -4.4934893
acc: 84.375 loss: -3.636011
acc: 84.375 loss: -3.3933883
acc: 79.6875 loss: -3.3341613
acc: 85.9375 loss: -3.8672633
acc: 95.3125 loss: -3.833325
acc: 92.1875 loss: -3.6657445
acc: 90.625 loss: -3.6764133
acc: 95.3125 loss: -4.020876
acc: 85.9375 loss: -3.436744
acc: 84.375 loss: -3.2405477
acc: 89.0625 loss: -3.8061929
acc: 90.625 loss: -3.9030101
acc: 90.625 loss: -3.568936
acc: 87.5 loss: -3.8073676